In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 

headers = {'User Agent':'Mozilla/5.0'}

players = ['tyler-ulis', 'henry-ellenson', 'malachi-richardson', 'joel-bolomboy','ben-simmons', 'brandon-ingram', 
           'jaylen-brown', 'kris-dunn', 'buddy-hield', 'jamal-murray', 'domantas-sabonis', 'marquese-chriss', 
           'taurean-prince', 'denzel-valentine', 'malik-beasley', 'caris-levert', 'pascal-siakam', 'dejounte-murray', 
           'fred-van-vleet', 'danuel-house', 'dorian-finney-smith', 'wade-baldwin','deandre-bembry', 'brice-johnson',
           'skal-labissiere', 'damian-jones', 'deyonta-davis', 'cheick-diallo', 'malcolm-brogdon', 'chinanu-onuaku', 
           'patrick-mccaw', 'diamond-stone', 'stephen-zimmermanjr', 'isaiah-whitehead', 'demetrius-jackson-2', 
           'aj-hammonds', 'jake-layman', 'michael-gbinije','georges-niang', 'ben-bentil', 'kay-felder', 
           'marcus-paige', 'daniel-hamilton', 'abdel-nader', 'isaiah-cousins', 'tyrone-wallace', 'alex-caruso', 
           'bryn-forbes', 'derrick-jonesjr', 'kevin-ferrell']
    
player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2016')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        playerlist.append(player)
        
    except:
        continue
    try:
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2016'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/30))*2.95), 2)
        year_list.append(year)
        
    except:
        age_list.append("No data")
        year_list.append(.85)

year_list[0] = .95
age_list[0] = 20.5
year_list[1] = 1.02
age_list[1] = 19.4
year_list[2] = .95
age_list[2] = 20.5
year_list[3] = .83
age_list[3] = 22.4

table.insert(0, "Name", playerlist)
table.insert(2, "Year", year_list)
table.insert(2, "Age", age_list)
table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

table["Player Grade"] = ((table['PTS']) + (table['TRB']*1.25) + (table['AST']*2) +
(table['BLK']*2) + (table['STL']*3) + (table['3P']*2) + (table['3PA']) + (table['SOS']/2)) * table['Year']

table["Player Grade"] = table["Player Grade"]*1.75
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['TOV']
del table['PF']
table

,Name,Season,Age,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,PTS,,Player Grade
1,Brandon Ingram,2015-16,18.8,Duke,10.59,36,34.6,.442,2.2,5.4,0.410,0.682,6.8,2.0,1.1,1.4,17.3,,98.2
2,Jamal Murray,2015-16,19.4,Kentucky,8.84,36,35.2,.454,3.1,7.7,0.408,0.783,5.2,2.2,1.0,0.3,20.0,,96.1
3,Ben Simmons,2015-16,20.0,LSU,7.00,33,34.9,.560,0.0,0.1,0.333,0.670,11.8,4.8,2.0,0.8,19.2,,93.9
4,Tyler Ulis,2015-16,20.5,Kentucky,8.84,35,36.8,.434,1.6,4.6,0.344,0.856,3.0,7.0,1.5,0.1,17.3,,86.4
5,Dejounte Murray,2015-16,19.8,Washington,8.12,34,33.5,.416,1.0,3.5,0.288,0.663,6.0,4.4,1.8,0.3,16.1,,83.9
6,Denzel Valentine,2015-16,22.6,Michigan State,6.94,31,33.0,.462,3.4,7.5,0.444,0.853,7.5,7.8,1.0,0.2,19.2,,83.5
7,Isaiah Whitehead,2015-16,21.3,Seton Hall,8.28,34,32.3,.379,2.4,6.5,0.365,0.760,3.6,5.1,1.2,1.4,18.2,,82.4
8,Henry Ellenson,2015-16,19.4,Marquette,5.57,33,33.5,.446,0.9,3.2,0.288,0.749,9.7,1.8,0.8,1.5,17.0,,81.9
9,Patrick Mccaw,2015-16,20.7,UNLV,3.17,33,33.7,.465,2.1,5.6,0.366,0.774,5.1,3.9,2.5,0.4,14.7,,77.3
10,Ben Bentil,2015-16,21.3,Providence,8.46,35,34.2,.462,1.5,4.5,0.329,0.782,7.7,1.1,0.9,1.0,21.1,,74.3
